In [ ]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog,messagebox
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime,timedelta
from tqdm import tqdm, tqdm_notebook
from statsmodels.tsa.holtwinters import SimpleExpSmoothing,ExponentialSmoothing
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA,ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score,mean_squared_error
from sklearn import linear_model,svm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import xgboost as xgb
import lightgbm as lgb
import math
import webbrowser
import warnings
warnings.filterwarnings("ignore")
class Start:
    def __init__(self,master):
        def callback(url):
            webbrowser.open_new(url)
        self.frame1=Frame(master, width=master.winfo_screenwidth(), height=master.winfo_screenheight(),bg="white")
        self.startframe=Frame(master,width=master.winfo_screenwidth(), height=master.winfo_screenheight(),bg="white")
        for frames in (self.frame1,self.startframe):
            frames.grid(row=0,column=0) 
        
        self.heading = Label(self.startframe, text="STOCK PRICE PREDICTION", font=("times", 30),bg="white").place(
            x=w//3, y=50)
        self.sc=Label(self.startframe, text="Enter The Stock Code: ", font=("times", 20),bg="white").place(x=50, y=150)
        self.sc1=Entry(self.startframe, font=("times", 20),bg="white",textvariable=stockcode).place(x=450, y=150)
        self.sc2=Label(self.startframe, text="search for stock codes", font=("times",12),bg="white",fg="blue", cursor="hand2")
        self.sc2.place(x=800, y=150)
        self.sc2.bind("<Button-1>", lambda e: callback("https://finance.yahoo.com/most-active"))
        self.d1=Label(self.startframe, text="Select The Algorithm: ", font=("times", 20),bg="white").place(x=50, y=250)
        self.d2=OptionMenu(self.startframe,alg,*alg_list)
        self.d2.place(x=450,y=250)
        self.d2.config(bg = "red",font=("times", 20),fg="white")
        self.d2['menu'].config(bg = "white",font=("times", 20))
        def opendata():
            DfG = yf.download(stockcode.get(), '2009-01-01',today, auto_adjust=True)
            DfG.to_csv('stock.csv',index=True)
            os.system("stock.csv")
        self.choosebutton1 = Button(self.startframe, text="View Dataset", font=("times", 20), bg="red", fg="white",
                                   padx=2,
                                   pady=2,
                                   command=opendata).place(x=900, y=500)
        self.choosebutton = Button(self.startframe, text="Train", font=("times", 20), bg="red", fg="white",
                                   padx=2,
                                   pady=2,
                                   command=self.train).place(x=1100, y=500)
        

    def switch_frame(self,next):
        next.tkraise()
    def train(self):
        def create_time_features(df,target=None):
            df['Date'] = df.index
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
            df['dayofweek'] = df['Date'].dt.dayofweek
            df['quarter'] = df['Date'].dt.quarter
            df['month'] = df['Date'].dt.month
            df['year'] = df['Date'].dt.year
            df['dayofyear'] = df['Date'].dt.dayofyear
            df['sin_day'] = np.sin(df['dayofyear'])
            df['cos_day'] = np.cos(df['dayofyear'])
            df['dayofmonth'] = df['Date'].dt.day
            df['weekofyear'] = df['Date'].dt.weekofyear
            X = df.drop(['Date'],axis=1)
            if target:
                y = df[target]
                X = X.drop([target],axis=1)
                return X, y

            return X
        DfG = yf.download(stockcode.get(), '2009-01-01',today, auto_adjust=True)
        DfG.to_csv('stock.csv',index=True)
        air_pollution=pd.read_csv('stock.csv')
        air_pollution.drop(['Volume'],axis=1,inplace=True)
        air_pollution.drop(['High'],axis=1,inplace=True)
        air_pollution.drop(['Open'],axis=1,inplace=True)
        air_pollution.drop(['Low'],axis=1,inplace=True)
        air_pollution['Date']=pd.to_datetime(air_pollution['Date'])
        air_pollution.set_index('Date', inplace=True)
        training_size=int(len(air_pollution)*0.80)
        test_size=len(air_pollution)-training_size
        df_training,df_test=air_pollution[0:training_size],air_pollution[training_size:len(air_pollution)]
        if alg.get()=="AR":
            index = len(df_training)
            yhat = list()
            for t in tqdm(range(len(df_test.Close))):
                temp_train = air_pollution[:len(df_training)+t]
                model = AR(temp_train.Close)
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                yhat = yhat + [predictions]
            yhat = pd.concat(yhat)
            a=r2_score(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1])
            b=math.sqrt(mean_squared_error(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1]))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="ARMA":
            index = len(df_training)
            yhat = list()
            for t in tqdm(range(len(df_test.Close))):
                temp_train = air_pollution[:len(df_training)+t]
                model = ARMA(temp_train.Close,order=(0, 1))
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                yhat = yhat + [predictions]
            yhat = pd.concat(yhat)
            a=r2_score(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1])
            b=math.sqrt(mean_squared_error(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1]))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="ARIMA":
            index = len(df_training)
            yhat = list()
            for t in tqdm(range(len(df_test.Close))):
                temp_train = air_pollution[:len(df_training)+t]
                model = ARIMA(temp_train.Close,order=(1,0,0))
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                yhat = yhat + [predictions]
            yhat = pd.concat(yhat)
            a=r2_score(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1])
            b=math.sqrt(mean_squared_error(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1]))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="SARIMAX":
            index = len(df_training)
            yhat = list()
            for t in tqdm(range(len(df_test.Close))):
                temp_train = air_pollution[:len(df_training)+t]
                model = SARIMAX(temp_train.Close,order=(1,0,0),seasonal_order=(0, 0, 0, 3))
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                yhat = yhat + [predictions]
            yhat = pd.concat(yhat)
            a=r2_score(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1])
            b=math.sqrt(mean_squared_error(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1]))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="SimpleExpSmoothing":
            index = len(df_training)
            yhat = list()
            for t in tqdm(range(len(df_test.Close))):
                temp_train = air_pollution[:len(df_training)+t]
                model = SimpleExpSmoothing(temp_train.Close)
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                yhat = yhat + [predictions]
            yhat = pd.concat(yhat)
            a=r2_score(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1])
            b=math.sqrt(mean_squared_error(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1]))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="ExpSmoothing":
            index = len(df_training)
            yhat = list()
            for t in tqdm(range(len(df_test.Close))):
                temp_train = air_pollution[:len(df_training)+t]
                model = ExponentialSmoothing(temp_train.Close)
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                yhat = yhat + [predictions]
            yhat = pd.concat(yhat)
            a=r2_score(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1])
            b=math.sqrt(mean_squared_error(df_test.Close[:len(df_test)-1], yhat.values[:len(df_test)-1]))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="BayesianRidge":
            X_train_df, y_train = create_time_features(df_training, target='Close')
            X_test_df, y_test = create_time_features(df_test, target='Close')
            scaler = StandardScaler()
            scaler.fit(X_train_df)
            X_train = scaler.transform(X_train_df)
            X_test = scaler.transform(X_test_df)
            reg = linear_model.BayesianRidge()
            reg.fit(X_train, y_train)
            yhat = reg.predict(X_test)
            a=r2_score(df_test.Close, yhat)
            b=math.sqrt(mean_squared_error(df_test.Close, yhat))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="Lasso":
            X_train_df, y_train = create_time_features(df_training, target='Close')
            X_test_df, y_test = create_time_features(df_test, target='Close')
            scaler = StandardScaler()
            scaler.fit(X_train_df)
            X_train = scaler.transform(X_train_df)
            X_test = scaler.transform(X_test_df)
            reg = linear_model.Lasso(alpha=0.1)
            reg.fit(X_train, y_train)
            yhat = reg.predict(X_test)
            a=r2_score(df_test.Close, yhat)
            b=math.sqrt(mean_squared_error(df_test.Close, yhat))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="RandomForestRegressor":
            X_train_df, y_train = create_time_features(df_training, target='Close')
            X_test_df, y_test = create_time_features(df_test, target='Close')
            scaler = StandardScaler()
            scaler.fit(X_train_df)
            X_train = scaler.transform(X_train_df)
            X_test = scaler.transform(X_test_df)
            reg = RandomForestRegressor(max_depth=2, random_state=0)
            reg.fit(X_train, y_train)
            yhat = reg.predict(X_test)
            a=r2_score(df_test.Close, yhat)
            b=math.sqrt(mean_squared_error(df_test.Close, yhat))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="XGBRegressor":
            X_train_df, y_train = create_time_features(df_training, target='Close')
            X_test_df, y_test = create_time_features(df_test, target='Close')
            scaler = StandardScaler()
            scaler.fit(X_train_df)
            X_train = scaler.transform(X_train_df)
            X_test = scaler.transform(X_test_df)
            reg = xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=1000)
            reg.fit(X_train, y_train)
            yhat = reg.predict(X_test)
            a=r2_score(df_test.Close, yhat)
            b=math.sqrt(mean_squared_error(df_test.Close, yhat))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="LGBMRegressor":
            X_train_df, y_train = create_time_features(df_training, target='Close')
            X_test_df, y_test = create_time_features(df_test, target='Close')
            scaler = StandardScaler()
            scaler.fit(X_train_df)
            X_train = scaler.transform(X_train_df)
            X_test = scaler.transform(X_test_df)
            reg = lgb.LGBMRegressor()
            reg.fit(X_train, y_train)
            yhat = reg.predict(X_test)
            a=r2_score(df_test.Close, yhat)
            b=math.sqrt(mean_squared_error(df_test.Close, yhat))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="SVM":
            X_train_df, y_train = create_time_features(df_training, target='Close')
            X_test_df, y_test = create_time_features(df_test, target='Close')
            scaler = StandardScaler()
            scaler.fit(X_train_df)
            X_train = scaler.transform(X_train_df)
            X_test = scaler.transform(X_test_df)
            reg = svm.SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
            reg.fit(X_train, y_train)
            yhat = reg.predict(X_test)
            a=r2_score(df_test.Close, yhat)
            b=math.sqrt(mean_squared_error(df_test.Close, yhat))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        if alg.get()=="KNeighborsRegressor":
            X_train_df, y_train = create_time_features(df_training, target='Close')
            X_test_df, y_test = create_time_features(df_test, target='Close')
            scaler = StandardScaler()
            scaler.fit(X_train_df)
            X_train = scaler.transform(X_train_df)
            X_test = scaler.transform(X_test_df)
            reg = KNeighborsRegressor(n_neighbors=2)
            reg.fit(X_train, y_train)
            yhat = reg.predict(X_test)
            a=r2_score(df_test.Close, yhat)
            b=math.sqrt(mean_squared_error(df_test.Close, yhat))
            rmse.set(str(b))
            r2.set(str(a))
            pv1.set("")
            pv.set("")
        self.switch_frame(self.frame1)
        self.heading1 = Label(self.frame1, text="RESULTS", font=("times", 30),bg="white").place(
            x=w// 3, y=50)
        self.rm1=Label(self.frame1,textvariable=rmse1,font=("times", 20),bg="white").place(x=50, y=150)
        rmse1.set("RMSE METRIC FOR "+alg.get()+" ALGORITHM: ")
        self.rm2=Label(self.frame1, textvariable=rmse, font=("times", 20),bg="white").place(x=750, y=150)
        self.r21=Label(self.frame1, textvariable=r21, font=("times", 20),bg="white").place(x=50, y=250)
        r21.set("R2 METRIC FOR "+alg.get()+" ALGORITHM: ")
        self.r22=Label(self.frame1, textvariable=r2, font=("times", 20),bg="white").place(x=750, y=250)
        goback = Button(self.frame1, text='GoBack',font=("times", 20), bg="red", fg="white",padx=2,pady=2,command=lambda: self.switch_frame(self.startframe))
        goback.place(x=750, y=350)
        def predict():
            if alg.get()=="SimpleExpSmoothing":
                temp_train = air_pollution[:]
                model = SimpleExpSmoothing(temp_train.Close)
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(list(predictions)[-1]))
            if alg.get()=="ExpSmoothing":
                temp_train = air_pollution[:]
                model = ExponentialSmoothing(temp_train.Close)
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(list(predictions)[-1]))
            if alg.get()=="AR":
                temp_train = air_pollution[:]
                model = AR(temp_train.Close)
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(list(predictions)[-1]))
            if alg.get()=="ARMA":
                temp_train = air_pollution[:]
                model = ARMA(temp_train.Close,order=(0, 1))
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(list(predictions)[-1]))
            if alg.get()=="ARIMA":
                temp_train = air_pollution[:]
                model = ARIMA(temp_train.Close,order=(1,0,0))
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(list(predictions)[-1]))
            if alg.get()=="SARIMAX":
                temp_train = air_pollution[:]
                model = SARIMAX(temp_train.Close,order=(1,0,0),seasonal_order=(0, 0, 0, 3))
                model_fit = model.fit()
                predictions = model_fit.predict(start=len(temp_train), end=len(temp_train))
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(list(predictions)[-1]))
            if alg.get()=="BayesianRidge":
                df1=pd.DataFrame([[pd.to_datetime(today),0.00]],columns=['Date','Close'])
                df1.set_index('Date', inplace=True)
                X_train_df, y_train = create_time_features(air_pollution, target='Close')
                X_test_df, y_test = create_time_features(df1, target='Close')
                scaler = StandardScaler() 
                scaler.fit(X_train_df)
                X_train = scaler.transform(X_train_df)
                X_test = scaler.transform(X_test_df)
                reg = linear_model.BayesianRidge()
                reg.fit(X_train, y_train)
                predictions=reg.predict(X_test)
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(predictions[0]))
            if alg.get()=="Lasso":
                df1=pd.DataFrame([[pd.to_datetime(today),0.00]],columns=['Date','Close'])
                df1.set_index('Date', inplace=True)
                X_train_df, y_train = create_time_features(air_pollution, target='Close')
                X_test_df, y_test = create_time_features(df1, target='Close')
                scaler = StandardScaler() 
                scaler.fit(X_train_df)
                X_train = scaler.transform(X_train_df)
                X_test = scaler.transform(X_test_df)
                reg = linear_model.Lasso(alpha=0.1)
                reg.fit(X_train, y_train)
                predictions=reg.predict(X_test)
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(predictions[0]))
            if alg.get()=="RandomForestRegressor":
                df1=pd.DataFrame([[pd.to_datetime(today),0.00]],columns=['Date','Close'])
                df1.set_index('Date', inplace=True)
                X_train_df, y_train = create_time_features(air_pollution, target='Close')
                X_test_df, y_test = create_time_features(df1, target='Close')
                scaler = StandardScaler() 
                scaler.fit(X_train_df)
                X_train = scaler.transform(X_train_df)
                X_test = scaler.transform(X_test_df)
                reg = RandomForestRegressor(max_depth=2, random_state=0)
                reg.fit(X_train, y_train)
                predictions=reg.predict(X_test)
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(predictions[0]))
            if alg.get()=="XGBRegressor":
                df1=pd.DataFrame([[pd.to_datetime(today),0.00]],columns=['Date','Close'])
                df1.set_index('Date', inplace=True)
                X_train_df, y_train = create_time_features(air_pollution, target='Close')
                X_test_df, y_test = create_time_features(df1, target='Close')
                scaler = StandardScaler() 
                scaler.fit(X_train_df)
                X_train = scaler.transform(X_train_df)
                X_test = scaler.transform(X_test_df)
                reg = xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=1000)
                reg.fit(X_train, y_train)
                predictions=reg.predict(X_test)
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(predictions[0]))
            if alg.get()=="LGBMRegressor":
                df1=pd.DataFrame([[pd.to_datetime(today),0.00]],columns=['Date','Close'])
                df1.set_index('Date', inplace=True)
                X_train_df, y_train = create_time_features(air_pollution, target='Close')
                X_test_df, y_test = create_time_features(df1, target='Close')
                scaler = StandardScaler() 
                scaler.fit(X_train_df)
                X_train = scaler.transform(X_train_df)
                X_test = scaler.transform(X_test_df)
                reg = lgb.LGBMRegressor()
                reg.fit(X_train, y_train)
                predictions=reg.predict(X_test)
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(predictions[0]))
            if alg.get()=="SVM":
                df1=pd.DataFrame([[pd.to_datetime(today),0.00]],columns=['Date','Close'])
                df1.set_index('Date', inplace=True)
                X_train_df, y_train = create_time_features(air_pollution, target='Close')
                X_test_df, y_test = create_time_features(df1, target='Close')
                scaler = StandardScaler() 
                scaler.fit(X_train_df)
                X_train = scaler.transform(X_train_df)
                X_test = scaler.transform(X_test_df)
                reg = svm.SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
                reg.fit(X_train, y_train)
                predictions=reg.predict(X_test)
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(predictions[0]))
            if alg.get()=="KNeighborsRegressor":
                df1=pd.DataFrame([[pd.to_datetime(today),0.00]],columns=['Date','Close'])
                df1.set_index('Date', inplace=True)
                X_train_df, y_train = create_time_features(air_pollution, target='Close')
                X_test_df, y_test = create_time_features(df1, target='Close')
                scaler = StandardScaler() 
                scaler.fit(X_train_df)
                X_train = scaler.transform(X_train_df)
                X_test = scaler.transform(X_test_df)
                reg =KNeighborsRegressor(n_neighbors=2)
                reg.fit(X_train, y_train)
                predictions=reg.predict(X_test)
                pv1.set("PREDICTED VALUE FOR NEXT DAY: ")
                pv.set(str(predictions[0]))
            if pv.get()!="":
                self.f1=Label(self.frame1, text="Select The File Format To Save The Predicted Value: ", font=("times", 20),bg="white").place(x=100, y=550)
                self.f3=OptionMenu(self.frame1,ff,*f_format)
                self.f3.place(x=750,y=550)
                self.f3.config(bg = "red",font=("times", 20),fg="white")
                self.f3['menu'].config(bg = "white",font=("times", 20))
                def s_file():
                    if ff.get()=='.txt':
                        with open("C:/Users/vinu/Desktop/out.txt",'w') as file:
                            file.write(pv1.get()+pv.get())
                            file.close()
                    if ff.get()=='.doc':
                        with open("C:/Users/vinu/Desktop/out.doc",'w') as file:
                            file.write(pv1.get()+pv.get())
                            file.close()
                self.f2 = Button(self.frame1, text='Save',font=("times", 20), bg="red", fg="white",padx=2,pady=2,command=s_file)
                self.f2.place(x=500, y=600)
        self.p3=Label(self.frame1, text="",textvariable=pv1, font=("times", 20),bg="white").place(x=100, y=500)
        self.p2=Label(self.frame1, text="",textvariable=pv, font=("times", 20),bg="white").place(x=580, y=500)
        self.p1 = Button(self.frame1, text='Predict',font=("times", 20), bg="red", fg="white",padx=2,pady=2,command=predict)
        self.p1.place(x=500, y=350)
root=Tk()
w=root.winfo_screenwidth()
h=root.winfo_screenheight()
root.geometry(str(w)+"x"+str(h))
alg_list=["SimpleExpSmoothing","ExpSmoothing","AR","ARMA","ARIMA","SARIMAX","BayesianRidge","Lasso","RandomForestRegressor","XGBRegressor"
          ,"LGBMRegressor","SVM","KNeighborsRegressor"]
f_format=[".txt",".doc"]
ff=StringVar()
ff.set(".txt")
stockcode=StringVar()
alg=StringVar()
rmse=StringVar()
r2=StringVar()
rmse1=StringVar()
r21=StringVar()
pv=StringVar()
pv1=StringVar()
today = str(datetime.now()+timedelta(1)).split(' ')[0]
alg.set("SimpleExpSmoothing")
start=Start(root)
root.mainloop()